In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch.utils.data as D
import cv2
from PIL import Image
from torchvision import transforms,models,datasets
from fastai.vision import *

transform = transforms.Compose([
                                transforms.CenterCrop(10),
                                transforms.Resize(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485],
                                 std=[0.229])
])
trainset = datasets.ImageFolder(root='/kaggle/input/special5/spl5/',transform=transform)
print(len(trainset.classes))
classesDict = {idx:i for idx,i in enumerate(trainset.classes)}
path = '/kaggle/input/special5/'
iB = ImageDataBunch.from_folder(path=path,
                   train = 'spl5',
                   seed = 42,
                   valid_pct = 0.2,
                   size=224,
                    bs=32,
                   ds_tfms=get_transforms(do_flip=True,
                                      max_warp=0,
                                      max_rotate=0,
                                      max_lighting=0,
                                      p_affine=0,
                                      xtra_tfms=[crop_pad()]))
print(iB)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
learn3 = cnn_learner(data=iB,base_arch=models.densenet201,model_dir='/tmp/models',metrics=[accuracy])
mod = learn3.model

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import torch
learn3.lr_find()
learn3.recorder.plot(suggestion=True)

In [ ]:
learn3.unfreeze()
learn3.fit_one_cycle(10,slice(5e-03))

In [ ]:
learn3.show_results()
learn3.save('model',return_path=True)

In [ ]:
# print(f'Training Accuracy:{round(100*correct/total,2)}%')
# # print(f'Validation Accuracy:{round(100*t_cor/t_tot,2)}%')
import matplotlib.pyplot as plt
import cv2
import numpy as np
import matplotlib.image as mpimg
%matplotlib inline
face_cascade = cv2.CascadeClassifier('/kaggle/input/detectors/haarcascade_frontalface_default.xml')
image = cv2.imread('/kaggle/input/tester/IMG-20171130-WA0012.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
faces = face_cascade.detectMultiScale(image, 1.2, 2)
image_with_detections = image.copy()
for (x,y,w,h) in faces:
    cv2.rectangle(image_with_detections,(x,y),(x+w,y+h),(224,0,0),3) 

fig = plt.figure(figsize=(9,9))

plt.imshow(image_with_detections)
mod = learn3.model


In [ ]:
# mod.load_state_dict(torch.load('/tmp/models/model.pth'))

image_copy = np.copy(image)
att = []
# loop over the detected faces from your haar cascade
for idx,(x,y,w,h) in enumerate(faces):
    # Select the region of interest that is the face in the image 
    roi = image_copy[y:y+h, x:x+w]
    
    ## TODO: Convert the face region from RGB to grayscale
    roi = roi/255.0
    ## TODO: Normalize the grayscale image so that its color range falls in [0,1] instead of [0,255]
    ## TODO: Rescale the detected face to be the expected square size for your CNN (224x224, suggested)
    h,w = roi.shape[:2]   
    output_size = 224
    if h>w:
        new_h, new_w = output_size*h/w, output_size
    else:
        new_h, new_w = output_size, output_size*w/h
    new_h, new_w = int(new_h), int(new_w)

    img = cv2.resize(roi, (new_w, new_h))
    

    
    ## TODO: Reshape the numpy image shape (H x W x C) into a torch image shape (C x H x W)
    img = img.reshape(img.shape[0], img.shape[1], 3)
    img = img.transpose((2, 0, 1))
    img = torch.from_numpy(img)
    img.unsqueeze_(0)
    
    ## TODO: Make facial keypoint predictions using your loaded, trained network 
    mod.eval()
    mod = mod.double()
    img = img.to(device)
    out = mod(img)
    out = out.cpu()
    out = out.detach().numpy()
    out = np.exp(out)
    li = out.tolist()
    ind = li[0].index(max(li[0]))
    att.append(classesDict[ind])
    print(classesDict[ind])
    img = img.cpu()
    img = img.reshape(224,224,3)
    plt.imshow(np.squeeze(img))
    plt.axis('off')
plt.show()


In [ ]:
df = pd.DataFrame(att, columns=["attendees"])
df.to_csv('list.csv', index=False)